# <a href="https://mipt-stats.gitlab.io/courses/ad_mipt.html">Phystech@DataScience</a>
## Задание 5

**Правила:**

* Выполненную работу нужно отправить телеграм-боту `@miptstats_pds_bot`.
* Дедлайн **10 апреля в 23:00**. После дедлайна работы не принимаются кроме случаев наличия уважительной причины.
* Прислать нужно ноутбук в формате `ipynb` и все фотографии, если пишете теоретическую часть от руки.
* Решения, размещенные на каких-либо интернет-ресурсах, не принимаются. Публикация решения может быть приравнена к предоставлении возможности списать.
* Для выполнения задания используйте этот ноутбук в качестве основы, ничего не удаляя из него.

-----

*Замечания.* Теоретические решения можно оформить
* в $\LaTeX$-формате в ноутбуке;
* написать от руки и прикрепить к ноутбуку;
* написать от руки и выслать боту.  

Во втором случае также **важно** "вшить" фото в ноутбук. Сделать это можно с помощью Edit -> Insert Image в Jupyter или с помощью кнопки "Вставить изображение" в Colab. Следите за размером итогового файла.

Фотографии принимаются только в хорошем качестве, **следите за освещением и почерком**. На фотографиях также указывайте номера задач.

-----

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as sps

import matplotlib.pyplot as plt
import seaborn as sns

red = '#FF3300'
blue = '#0099CC'
green = '#00CC66'

from statsmodels.sandbox.stats.multicomp import multipletests
from tqdm.notebook import tqdm

%matplotlib inline


#Теоретическая часть

## p-value (основной поток)

### Задача 1. 
 Для всех пунктов задач 3 и 4 прошлого домашнего задания выпишите формулу для p-value  $p(t) = <...>$  в виде кода на `scipy`, где $t$ - реализация статистики вашего критерия, т.е. $t = T(x), x$ &mdash; реализация выборки, $T(X)$ &mdash; статистика. 

Для задачи 4 также посчитайте численные значения для обоих случаев p-value для данного в задаче существа. Какие гипотезы отклоняются? 

Вычисления можно выполнить в Питоне по приведенным вами формулам. 

**Решение:**

In [20]:
#3a
theta = 3
theta0 = 0
x = sps.norm(loc=theta).rvs(100)
t = x.sum(); n = len(x)
p = sps.norm(loc=n*theta0, scale=n**0.5).cdf(t)
print(p)

1.0


In [21]:
#3b Аналологично
p = sps.norm(loc=n*theta0, scale=n**0.5).sf(t)
print(p)

8.763568550500895e-196


In [76]:
#4a H0: пёсик vs H1: единорог
alpha = 0.05
x = 6.66
p = sps.gamma(a=2, scale=7/6).sf(x)
print("p-value =", p)
print("Отвергаем:", alpha > p)

p-value = 0.022255070646549243
Отвергаем: True


In [77]:
#4б H0: единорог vs H1: пёсик
x = 6.66
p = sps.gamma(a=3, scale=44/5).cdf(x)
print("p-value =", p)
print("Отвергаем:", alpha > p)

p-value = 0.041416397470335854
Отвергаем: True


 **Вывод:** написали p-value, в 4 задаче в обоих случаях гипотезы отвергаются (в первом случае p-value меньше, значит отвергается более достоверно).

## Множественная проверка гипотез (основной поток)


### Задача 2.
 Пусть $X_1, ...,  X_n$ --- выборка из неизвестного распределения $\mathsf{P}$. Для проверки гипотезы $\mathsf{H}_0\ vs.\ \mathsf{H}_1$ было решено использовать три различных критерия. Соответствующие p-value равны 0.00001, 0.7361, 0.0482. Какое должно быть принято решение об отвержении гипотезы $\mathsf{H}_0$ на уровне значимости 0.05? *Подсказка: используйте любой подходящий метод МПГ, далее делайте вывод об отвержении/не отвержении, поясните свой вывод.*

**Решение:**

In [79]:
p = [0.00001, 0.7361, 0.0482]
res = multipletests(p, alpha=0.05, method='h', is_sorted=False, returnsorted=False)
print('Отвергаем: ', *res[0])
print('Скорректированные p-value:', res[1])

Отвергаем:  True False False
Скорректированные p-value: [3.000e-05 7.361e-01 9.640e-02]


**Вывод:** для проверки гипотезы каждым критерием получаем свою нулевую гипотезу, применяем МПГ. С одной стороны, даже с исправленным уровнем значимости одна из проверок даёт отвержение, с другой стороны p-value у неё очень маленький, что показывает, что критерий очень мощный, значит . может расти ошибка первого рода, второй критерий очень слабомощный, третий - врое нормальный и он не даёт отвержения. Значит в принципе гипотезу можно не отвергать.  

# Практическая часть

## Множественная проверка гипотез


### Задача 3 (все потоки).
Проведены эксперименты для оценки эффективности нескольких препаратов для снижения послеоперационной тошноты. Результаты экспериментов приведены в таблице ниже. При проведении эксперимента пациенты делились на группы случайным образом.

    
                            Количество пациентов  Количество случаев возникновения тошноты

		Плацебо                80                    45 

		Хлорпромазин           75                    26 
    
		Дименгидринат          85                    52 
    
		Пентобарбитал (100 мг) 67                    35 
    
		Пентобарбитал (150 мг) 85                    37 
    

 Проведите сравнение каждого препарата по эффективности по отношению к плацебо c использованием критерия Вальда (см. лекцию 5 и последующие). Какие методы МПГ, контролирующие FWER и FDR, можно использовать в данной задаче? Какие ответы можно получить для этих методов? В каждом случае приведите значения статистики критерия Вальда, p-value и скорректированные p-value. Поясните смысл p-value и множественной проверки гипотез в данной задаче. Оформите решение структурированно. 

In [29]:
data = pd.DataFrame([["Плацебо", 80, 45],
                     ["Хлорпромазин", 75, 26],
                     ["Дименгидринат", 85, 52],
                     ["Пентобарбитал (100 мг)", 67, 35],
                     ["Пентобарбитал (150 мг)", 85, 37]])

data.columns = ["Название", "Количество пациентов", "Количество случаев возникновения тошноты"]

In [30]:
data

,Название,Количество пациентов,Количество случаев возникновения тошноты
0,Плацебо,80,45
1,Хлорпромазин,75,26
2,Дименгидринат,85,52
3,Пентобарбитал (100 мг),67,35
4,Пентобарбитал (150 мг),85,37


In [97]:
#Запишем критерии
ave = data["Количество случаев возникновения тошноты"] / data["Количество пациентов"]

#Для каждого случая посчитаем соответствующую статистику
t = 1 - ave[1:] #доля людей без тошноты/
p0 = 1 - ave[0]
sizes = data["Количество пациентов"]
sigma = (t * (1 - t) / sizes[1:] + p0 * (1 - p0) / sizes[0]) ** 0.5
t = (t - p0) / sigma

#Считаем p-value
p = sps.norm().sf(t)
#Проверяем гипотезы
res = multipletests(p, alpha=0.05, method='hs', is_sorted=False, returnsorted=False)
rej = res[0]
p_cor = res[1]

res = multipletests(p, alpha=0.05, method='fdr_by', is_sorted=False, returnsorted=False)
rej_fdr = res[0]
p_cor_fdr = res[1]

ans = pd.DataFrame([
    [None for i in range(7)],
    [None for i in range(7)],
    [None for i in range(7)],
    [None for i in range(7)]   
])
ans.columns = ["Название", "Статистика Вальда", "p-value", "Скорректированные p-value по Холму-Шидаку", "Отвержение по Холму-Шидаку", 
              "Скорректированные p-value по Бенджамини-Иекутиели", "Отвержение по Бенджамини-Иекутиели"]

ans["Название"] = np.array(data["Название"][1:])
ans["Статистика Вальда"] = np.array(t)
ans["p-value"] = p
ans["Скорректированные p-value по Холму-Шидаку"] = p_cor
ans["Отвержение по Холму-Шидаку"] = rej
ans["Скорректированные p-value по Бенджамини-Иекутиели"] = p_cor_fdr
ans["Отвержение по Бенджамини-Иекутиели"] = rej_fdr
ans

,Название,Статистика Вальда,p-value,Скорректированные p-value по Холму-Шидаку,Отвержение по Холму-Шидаку,Скорректированные p-value по Бенджамини-Иекутиели,Отвержение по Бенджамини-Иекутиели
0,Хлорпромазин,2.764364,0.002852,0.011358,True,0.023764,True
1,Дименгидринат,-0.642987,0.739884,0.739884,False,1.000000,False
2,Пентобарбитал (100 мг),0.486428,0.313332,0.528487,False,0.870367,False
3,Пентобарбитал (150 мг),1.646605,0.049820,0.142137,False,0.207582,False


**Вывод:** было проведено исследование эффективности препартов от тошноты МПГ (он использовался, чтобы не накапливать групповую ошибку первого рода). Из всех препаратов достаточно эффективным покахал себя только хлорпромазин, для остальных отвергнуть гипотезу тождественности с плацебо не удалось.

Так как дозировка препарата, может очень непредсказуемо менять его дейтсвие, а иссследование проводилось на разных людях, статистики по препаратам можно считать независмыми. Применяя соответсвтующий метод контроля FWER и FDR (Шидака-Холма и Бенджамини-Иекутиели соответсвенно), опровергли  всё равно только гипотезу хлорпромазина. Посчитанное p-value позволяет убедиться в достаточно уверенном отвержении.

МПГ используется, чтобы не назвать эффективными препараты, таковыми не являющимися, потому что вероятность того, что тошноты не возникнет на большой выборке людей довольно высока.

p-value в данном случае показывает, с какой вероятностью в группе, где раздавали плацебо, будет больше людей без тошноты, чем в группе по данному препарату (по процентам).